# **Estrategia de trading con aprendizaje no supervisado**

- Descargar/Cargar datos de precios de acciones del SP500.
- Calcular diferentes características e indicadores de cada acción.
- Agregar a nivel mensual y filtrar los 150 valores más líquidos de cada mes.
- Calcular la rentabilidad mensual para diferentes horizontes temporales.
- Descargar Factores Fama-French y Calcular Rolling Factor Betas.
-  Para cada mes, se entrena un Algoritmo de Clustering de K-Means para agrupar activos similares en función de sus características.
- Para cada mes, seleccionamos los activos en función del cluster y armamos un portafolio basado en la optimización del Efficient Frontier max sharpe ratio.
- Visualize Portfolio returns and compare to SP500 returns.

# **Todos los paquetes necesarios:**

- Pandas, Numpy, Matplotlib, statsmodels, pandas_datareader, datetime, yfinance, sklearn, PyPortfolioOpt

In [ ]:
!pip install pandas_ta
!pip install PyPortfolioOpt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218907 sha256=5292c5fa1d8590437700152da50132ccb31309cd6e0dcf140f5160b916e0809d
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 kB 624.0 kB/s eta 0:00:00


# **1. Descargar/Cargar datos de precios de acciones del SP500**

In [ ]:
# Importación de bibliotecas necesarias
from statsmodels.regression.rolling import RollingOLS  # Para regresiones móviles
import pandas_datareader.data as web  # Para descargar datos financieros
import matplotlib.pyplot as plt  # Para visualización de gráficos
import statsmodels.api as sm  # Para modelos estadísticos
import pandas as pd  # Para manipulación de datos
import numpy as np  # Para operaciones numéricas
import datetime as dt  # Para manejo de fechas y horas
import yfinance as yf  # Para descargar datos de Yahoo Finance
import pandas_ta  # Para análisis técnico
import warnings  # Para manejar advertencias
warnings.filterwarnings('ignore')  # Ignorar advertencias para limpiar la salida

# Descarga la lista de empresas que forman parte del S&P 500 desde Wikipedia
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

# Reemplaza puntos en los símbolos de las acciones por guiones
# Esto es necesario porque Yahoo Finance utiliza guiones en lugar de puntos
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

# Crea una lista de todos los símbolos de acciones únicos del S&P 500
symbols_list = sp500['Symbol'].unique().tolist()

# Define la fecha final para la descarga de datos
end_date = '2023-09-27'

# Calcula la fecha de inicio (8 años antes de la fecha final)
start_date = pd.to_datetime(end_date)-pd.DateOffset(365*8)

# Descarga los datos históricos de precios de las acciones del S&P 500
# desde Yahoo Finance, para el rango de fechas definido
df = yf.download(tickers=symbols_list,
                 start=start_date,
                 end=end_date).stack()

# Establece los nombres de los índices para el DataFrame
# 'date' para la fecha y 'ticker' para el símbolo de la acción
df.index.names = ['date', 'ticker']

# Convierte los nombres de las columnas a minúsculas para estandarización
df.columns = df.columns.str.lower()

df

[*********************100%%**********************]  503 of 503 completed

ERROR:yfinance:
1 Failed download:
ERROR:yfinance:['VLTO']: Exception("%ticker%: Data doesn't exist for startDate = 1443499200, endDate = 1695787200")


adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.588037   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361622   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.716072   27.264999   28.377501   26.965000   28.207500   
           ABBV     37.024620   52.790001   54.189999   51.880001   53.099998   
           ABT      33.807270   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 YUM     123.426186  124.010002  124.739998  123.449997  124.239998   
           ZBH     112.216316  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZION     33.581326   33.990002   34.700001   33.840000   33.840000   
           ZTS     176.447540  176.869995  178.449997  176.270004  176.580002   

                        volume  
date       ticker               
2015-09-29 A         2252400.0  
           AAL       7478800.0  
           AAPL    293461600.0  
           ABBV     12842800.0  
           ABT      12287500.0  
...                        ...  
2023-09-26 YUM       1500600.0  
           ZBH       3610500.0  
           ZBRA       355400.0  
           ZION      1586100.0  
           ZTS       1463200.0  

[993179 rows x 6 columns]

<div><font color="deepgray">

*Datos Cargados y lo que representa cada columna,en el contexto de los precios de las acciones del S&P 500.*</font></div>
<div><font color="deepgray">

## **Columnas del DataFrame** </font></div>
<div><font color="deepgray">

1. **adj close:** Esta es la columna "Adjusted Close" (Cierre Ajustado). Representa el precio de cierre de la acción ajustado por cualquier acción corporativa como dividendos, divisiones de acciones (splits), y derechos. Es una medida importante porque refleja el valor real de una acción teniendo en cuenta estos factores, lo que proporciona una imagen más precisa para el análisis histórico.</font></div>
<div><font color="deepgray">


2. **close:** El precio de cierre ("Close") es el precio de la acción al final del día de trading. No está ajustado por acciones corporativas.</font></div>
<div><font color="deepgray">


3. **high:** El precio más alto ("High") al que se ha negociado la acción durante el día.</font></div>
<div><font color="deepgray">


4. **low:** El precio más bajo ("Low") al que se ha negociado la acción durante el día.</font></div>
<div><font color="deepgray">


5. **open:** El precio de apertura ("Open") es el precio al que se negoció la acción al inicio del día de trading.</font></div>
<div><font color="deepgray">


6. **volume:** El volumen representa el número total de acciones que se han negociado durante el día.</font></div>
<div><font color="deepgray">


## **Tickers** </font></div>
<div><font color="deepgray">

- Los tickers en tu DataFrame son los símbolos únicos asignados a cada compañía que cotiza en bolsa. Por ejemplo, "AAPL" es el ticker para Apple Inc., y "MSFT" para Microsoft Corporation. Estos tickers son utilizados por las bolsas de valores y los traders para identificar y realizar transacciones con las acciones de las empresas.</font></div>
<div><font color="deepgray">


## **Uso de los Datos** </font></div>
<div><font color="deepgray">

- Estos datos son fundamentales para el análisis de acciones. Por ejemplo, el precio de cierre ajustado se utiliza a menudo para calcular rentabilidades históricas, mientras que el volumen puede ser un indicador de la liquidez de la acción.</font></div>

<div><font color="deepgray">

- Los precios de apertura y cierre (ajustados y no ajustados) te dan una idea de la variación del precio de la acción durante el día y a lo largo del tiempo.</font></div>

<div><font color="deepgray">

- Los precios más altos y más bajos te ofrecen una visión de la volatilidad intradía de la acción.</font></div>

<div><font color="deepgray">

*Con estos datos, podes realizar una variedad de análisis financieros, como calcular rentabilidades, analizar tendencias, realizar pruebas de backtesting de estrategias de trading, entre otros.*</font></div>

# **2. Calcular las características y los indicadores técnicos de cada acción**

- Garman-Klass Volatility
- RSI
- Bollinger Bands
- ATR
- MACD
- Dollar Volume

\begin{equation}
\text{Garman-Klass Volatility} = \frac{(\ln(\text{High}) - \ln(\text{Low}))^2}{2} - (2\ln(2) - 1)(\ln(\text{Adj Close}) - \ln(\text{Open}))^2
\end{equation}

In [ ]:
# Calcula la volatilidad de Garman-Klass
df['garman_klass_vol'] = ((np.log(df['high']) - np.log(df['low'])) ** 2) / 2 - (2 * np.log(2) - 1) * ((np.log(df['adj close']) - np.log(df['open'])) ** 2)

# Calcula el Índice de Fuerza Relativa (RSI)
df['rsi'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.rsi(close=x, length=20))

# Calcula la banda baja de Bollinger
df['bb_low'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:, 0])

# Calcula la banda media de Bollinger
df['bb_mid'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:, 1])

# Calcula la banda alta de Bollinger
df['bb_high'] = df.groupby(level=1)['adj close'].transform(lambda x: pandas_ta.bbands(close=np.log1p(x), length=20).iloc[:, 2])

# Función para calcular el Rango Verdadero Medio (ATR)
def compute_atr(stock_data):
    atr = pandas_ta.atr(high=stock_data['high'], low=stock_data['low'], close=stock_data['close'], length=14)
    return atr.sub(atr.mean()).div(atr.std())

# Aplica la función compute_atr a cada grupo de acciones
df['atr'] = df.groupby(level=1, group_keys=False).apply(compute_atr)

# Función para calcular el MACD (Moving Average Convergence Divergence)
def compute_macd(close):
    macd = pandas_ta.macd(close=close, length=20).iloc[:, 0]
    return macd.sub(macd.mean()).div(macd.std())

# Aplica la función compute_macd a cada grupo de acciones
df['macd'] = df.groupby(level=1, group_keys=False)['adj close'].apply(compute_macd)

# Calcula el volumen en dólares
df['dollar_volume'] = (df['adj close'] * df['volume']) / 1e6

df

adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.588037   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361622   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.716072   27.264999   28.377501   26.965000   28.207500   
           ABBV     37.024620   52.790001   54.189999   51.880001   53.099998   
           ABT      33.807270   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 YUM     123.426186  124.010002  124.739998  123.449997  124.239998   
           ZBH     112.216316  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZION     33.581326   33.990002   34.700001   33.840000   33.840000   
           ZTS     176.447540  176.869995  178.449997  176.270004  176.580002   

                        volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2015-09-29 A         2252400.0         -0.000854        NaN       NaN   
           AAL       7478800.0         -0.000443        NaN       NaN   
           AAPL    293461600.0         -0.005441        NaN       NaN   
           ABBV     12842800.0         -0.049280        NaN       NaN   
           ABT      12287500.0         -0.008237        NaN       NaN   
...                        ...               ...        ...       ...   
2023-09-26 YUM       1500600.0          0.000037  36.057200  4.821521   
           ZBH       3610500.0          0.000224  31.893246  4.751923   
           ZBRA       355400.0          0.000133  29.494977  5.400991   
           ZION      1586100.0          0.000292  46.707775  3.527329   
           ZTS       1463200.0          0.000075  42.623483  5.161192   

                     bb_mid   bb_high       atr      macd  dollar_volume  
date       ticker                                                         
2015-09-29 A            NaN       NaN       NaN       NaN      71.148896  
           AAL          NaN       NaN       NaN       NaN     279.420098  
           AAPL         NaN       NaN       NaN       NaN    7253.218059  
           ABBV         NaN       NaN       NaN       NaN     475.499790  
           ABT          NaN       NaN       NaN       NaN     415.406831  
...                     ...       ...       ...       ...            ...  
2023-09-26 YUM     4.851489  4.881456  0.142547 -1.363696     185.213334  
           ZBH     4.791592  4.831260 -0.381708 -0.881067     405.157010  
           ZBRA    5.539167  5.677342 -0.057389 -1.600791      79.595386  
           ZION    3.582765  3.638202 -0.161699 -0.164625      53.263340  
           ZTS     5.220007  5.278822  0.651515 -1.188278     258.178041  

[993179 rows x 14 columns]

<div><font color="deepgray">

*Las nuevas columnas que has añadido a tu DataFrame son indicadores técnicos y métricas financieras que se utilizan comúnmente en el análisis de acciones y estrategias de trading. Cada uno de estos indicadores aporta información específica sobre el comportamiento de las acciones. Aquí te explico qué representa cada uno y por qué son útiles:*</font></div>
<div><font color="deepgray">

## **1. Garman-Klass Volatility**</font></div>
<div><font color="deepgray">

- **Qué es:** Una medida de la volatilidad de los precios de las acciones. Se calcula utilizando los precios de apertura, cierre, máximo y mínimo.
- **Por qué es importante:** Proporciona una estimación de la volatilidad del precio de una acción, lo cual es crucial para evaluar el riesgo y la incertidumbre del mercado.</font></div>
<div><font color="deepgray">

## **2. RSI (Relative Strength Index)**</font></div>
<div><font color="deepgray">

- **Qué es:** Un indicador de momento que mide la magnitud de los movimientos recientes de precios para evaluar condiciones de sobrecompra o sobreventa.
- **Por qué es importante:** Ayuda a identificar posibles puntos de reversión en el precio de las acciones al señalar condiciones extremas.</font></div>
<div><font color="deepgray">

## **3. Bollinger Bands (Bandas de Bollinger)**</font></div>
<div><font color="deepgray">

- **Qué es:** Un conjunto de tres líneas (banda superior, media e inferior) que se derivan de la media móvil y la desviación estándar de los precios.
- **Por qué es importante:** Las bandas de Bollinger se utilizan para evaluar la volatilidad y los niveles de sobrecompra o sobreventa. Una acción se considera sobrecomprada si el precio se acerca a la banda superior y sobrevendida si se acerca a la banda inferior.</font></div>
<div><font color="deepgray">

## **4. ATR (Average True Range)**</font></div>
<div><font color="deepgray">

- **Qué es:** Una medida de la volatilidad del mercado que toma en cuenta el rango completo de movimiento de precios (incluyendo gaps).
- **Por qué es importante:** El ATR es útil para entender la volatilidad de una acción y puede ayudar en la colocación de órdenes de stop-loss y en la gestión del riesgo.</font></div>
<div><font color="deepgray">

## **5. MACD (Moving Average Convergence Divergence)**</font></div>
<div><font color="deepgray">

- **Qué es:** Un indicador de tendencia que muestra la relación entre dos medias móviles de precios.
- **Por qué es importante:** El MACD ayuda a identificar cambios en la dirección, fuerza, momento y duración de una tendencia en el precio de una acción.</font></div>
<div><font color="deepgray">

## **6. Dollar Volume**</font></div>
<div><font color="deepgray">

- **Qué es:** El volumen total de dólares negociados, calculado como el precio de cierre ajustado multiplicado por el volumen de acciones negociadas.
- **Por qué es importante:** Proporciona una idea de la liquidez y el interés del mercado en una acción. Un volumen alto puede indicar una mayor confianza y un interés significativo en la acción.</font></div>

<div><font color="deepgray">

*Cada uno de estos indicadores aporta una perspectiva única sobre el comportamiento de las acciones y puede ser utilizado para tomar decisiones informadas en el trading y la inversión. La combinación de estos indicadores ofrece una visión más completa y matizada que si se considerara cada uno de manera aislada.*</font></div>

# **3. Agregar a nivel mensual y filtrar los 150 valores más líquidos de cada mes**

- Para reducir el tiempo de formación y experimentar con características y estrategias, convertimos los datos diarios de las empresas a la frecuencia de fin de mes.

In [ ]:
# Reestablece el índice del DataFrame y luego establece un índice múltiple basado en 'date' y 'ticker'
df = df.reset_index().set_index(['date', 'ticker'])

# Calcula el promedio mensual del 'dollar_volume' para cada 'ticker', reorganizando el DataFrame para tener 'ticker' como columnas
df_resampled = df['dollar_volume'].unstack('ticker').resample('M').mean().stack('ticker').to_frame('dollar_volume')

# Crea una lista de nombres de columnas que no son 'dollar_volume', 'volume', 'open', 'high', 'low', ni 'close'
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_volume', 'volume', 'open', 'high', 'low', 'close']]

# Combina dos DataFrames: 'df_resampled' y otro DataFrame creado a partir de 'df' que incluye solo las columnas en 'last_cols'
# Este segundo DataFrame se resamplea mensualmente, tomando el último valor de cada mes, y luego se aplica 'stack' para los tickers
# Concatena ambos DataFrames a lo largo del eje de las columnas y elimina las filas con valores faltantes
data = (pd.concat([df_resampled,
                   df.unstack()[last_cols].resample('M').last().stack('ticker')],
                  axis=1)).dropna()

# El resultado es un DataFrame 'data' que contiene el volumen promedio mensual en dólares y otros indicadores técnicos o métricas financieras para cada acción (ticker) y cada mes
data

dollar_volume   adj close       atr   bb_high    bb_low  \
date       ticker                                                            
2015-11-30 A          136.444135   39.152691 -1.033887  3.694119  3.549210   
           AAL        287.915795   39.429928  0.190822  3.827636  3.672028   
           AAPL      4034.583500   26.924868 -0.967900  3.370842  3.284211   
           ABBV       343.971761   41.160309 -0.526809  3.841588  3.745051   
           ABT        213.736347   38.669395 -1.064842  3.709289  3.665571   
...                          ...         ...       ...       ...       ...   
2023-09-30 YUM        176.806022  123.426186  0.142547  4.881456  4.821521   
           ZBH        192.575129  112.216316 -0.381708  4.831260  4.751923   
           ZBRA       105.780863  223.960007 -0.057389  5.677342  5.400991   
           ZION       100.279835   33.581326 -0.161699  3.638202  3.527329   
           ZTS        288.993350  176.447540  0.651515  5.278822  5.161192   

                     bb_mid  garman_klass_vol      macd        rsi  
date       ticker                                                   
2015-11-30 A       3.621664         -0.001810  0.567157  73.421493  
           AAL     3.749832         -0.000966 -0.418772  40.718939  
           AAPL    3.327527         -0.003119 -0.142790  55.537285  
           ABBV    3.793320         -0.053947  0.145677  49.376888  
           ABT     3.687430         -0.009962  0.335557  56.962541  
...                     ...               ...       ...        ...  
2023-09-30 YUM     4.851489          0.000037 -1.363696  36.057200  
           ZBH     4.791592          0.000224 -0.881067  31.893246  
           ZBRA    5.539167          0.000133 -1.600791  29.494977  
           ZION    3.582765          0.000292 -0.164625  46.707775  
           ZTS     5.220007          0.000075 -1.188278  42.623483  

[46894 rows x 9 columns]

- Calcular la media móvil de 5 años del volumen en dólares de cada acción antes del filtrado.

In [ ]:
# Calcula el promedio móvil de 5 años (60 meses) del 'dollar_volume' para cada acción (ticker)
# 'rolling' con una ventana de 60 meses y un mínimo de 12 meses para empezar a calcular
# 'mean().stack()' para reorganizar el DataFrame después de aplicar el promedio móvil
data['dollar_volume'] = (data.loc[:, 'dollar_volume'].unstack('ticker').rolling(5*12, min_periods=12).mean().stack())

# Calcula el rango de 'dollar_volume' dentro de cada fecha
# 'groupby('date')' agrupa los datos por fecha
# 'rank(ascending=False)' asigna un rango a cada valor de 'dollar_volume', con los valores más altos obteniendo rangos más bajos
data['dollar_vol_rank'] = (data.groupby('date')['dollar_volume'].rank(ascending=False))

# Filtra el DataFrame para mantener solo aquellos valores que están en los 150 primeros rangos de 'dollar_volume'
# Esto selecciona los 150 valores más líquidos (en términos de volumen en dólares) para cada mes
# 'drop(['dollar_volume', 'dollar_vol_rank'], axis=1)' elimina las columnas 'dollar_volume' y 'dollar_vol_rank' del DataFrame final
data = data[data['dollar_vol_rank']<150].drop(['dollar_volume', 'dollar_vol_rank'], axis=1)

# El resultado es un DataFrame 'data' que contiene solo los 150 valores más líquidos cada mes, junto con sus métricas financieras y técnicas
data

adj close       atr   bb_high    bb_low    bb_mid  \
date       ticker                                                       
2016-10-31 AAL      39.134323  0.402199  3.706314  3.604673  3.655493   
           AAPL     26.281519 -1.038688  3.354535  3.296770  3.325653   
           ABBV     41.009079 -0.893132  3.880188  3.771814  3.826001   
           ABT      34.630024 -1.035224  3.665095  3.564121  3.614608   
           ACN     104.350327 -0.996806  4.668056  4.644779  4.656418   
...                       ...       ...       ...       ...       ...   
2023-09-30 VRTX    351.690002  0.029799  5.879295  5.838959  5.859127   
           VZ       32.300549 -1.078816  3.563843  3.499366  3.531604   
           WFC      40.290813 -0.558742  3.790225  3.709473  3.749849   
           WMT     161.898697 -0.196379  5.113302  5.077929  5.095615   
           XOM     115.355156  0.601335  4.784477  4.704273  4.744375   

                   garman_klass_vol      macd        rsi  
date       ticker                                         
2016-10-31 AAL            -0.000176  1.131595  62.203510  
           AAPL           -0.002307 -0.195978  49.891057  
           ABBV           -0.041756 -0.760593  27.477672  
           ABT            -0.006476 -0.650888  38.008845  
           ACN            -0.004026 -0.135456  53.823729  
...                             ...       ...        ...  
2023-09-30 VRTX            0.000037  0.027907  52.406728  
           VZ             -0.000067 -0.350385  42.222474  
           WFC             0.000136 -0.282325  40.920284  
           WMT             0.000011  0.399459  54.722511  
           XOM             0.000068  1.400623  59.440184  

[12516 rows x 8 columns]

<div><font color="deepgray">

## **1. Promedio Móvil de 5 Años del Volumen en Dólares**</font></div>
<div><font color="deepgray">

**Qué es:** Calculaste el promedio móvil de 5 años del volumen en dólares para cada acción. Esto implica suavizar las fluctuaciones del volumen de negociación a lo largo del tiempo para obtener una visión más estable de la liquidez de cada acción.</font></div>

<div><font color="deepgray">

**Por qué es importante:** Este paso es crucial para identificar las acciones con mayor liquidez en el mercado. Un volumen en dólares alto y consistente sugiere una mayor facilidad para entrar y salir de posiciones en esas acciones, un factor clave en la toma de decisiones de inversión y trading.</font></div>

<div><font color="deepgray">

## **2. Clasificación del Volumen en Dólares**</font></div>

<div><font color="deepgray">

**Qué es:** Clasificaste las acciones en función de su volumen en dólares, ordenándolas de mayor a menor cada mes.</font></div>

<div><font color="deepgray">

**Por qué es importante:** Esta clasificación permite identificar las 150 acciones más líquidas cada mes. Al concentrarte en estas acciones, reduces el riesgo asociado con la baja liquidez y te enfocas en las acciones que son más relevantes en el mercado.</font></div>

<div><font color="deepgray">

## **3. Filtrado de las 150 Acciones Más Líquidas**</font></div>

<div><font color="deepgray">

**Qué es:** Filtraste el conjunto de datos para incluir solo las 150 acciones más líquidas según el ranking de volumen en dólares.</font></div>

<div><font color="deepgray">

**Por qué es importante:** Este filtrado asegura que tu análisis y estrategias de trading se centren en acciones con suficiente volumen de negociación, lo que es esencial para la ejecución efectiva de las operaciones y la minimización del impacto del mercado.</font></div>

<div><font color="deepgray">

## **Resultado Final**</font></div>

<div><font color="deepgray">

El resultado final es un DataFrame que contiene información sobre las 150 acciones más líquidas cada mes, junto con varios indicadores técnicos clave.</font></div>

<div><font color="deepgray">

## **Importancia de Estos Pasos**</font></div>

<div><font color="deepgray">
- Enfoque en Liquidez: Al centrarte en las 150 acciones más líquidas, tu análisis se basa en acciones con suficiente actividad de mercado.
- Análisis Técnico: Los indicadores seleccionados proporcionan insights valiosos para estrategias de trading y análisis de riesgo.
- Base para Estrategias de Trading: Esta información es crucial para desarrollar y probar estrategias de trading, ya que proporciona una base sólida para la toma de decisiones informadas.</font></div>

# **4. Calcular Rentabilidades Mensuales para diferentes horizontes temporales como características**

- Para captar la dinámica de las series temporales que reflejan, por ejemplo, patrones de impulso, calculamos los rendimientos históricos utilizando el método .pct_change(lag), es decir, los rendimientos durante varios periodos mensuales identificados por lags.

In [ ]:
def calculate_returns(df):
    # Define un umbral para identificar y limitar los valores atípicos en los rendimientos
    outlier_cutoff = 0.005

    # Define los horizontes temporales (lags) para los que se calcularán los rendimientos
    lags = [1, 2, 3, 6, 9, 12]

    # Itera sobre cada lag
    for lag in lags:
        # Calcula el rendimiento porcentual para el lag actual
        # .pct_change(lag) calcula el cambio porcentual en 'adj close' para el número de periodos especificado en 'lag'
        # .pipe(...) aplica una función que limita los valores extremos basándose en cuantiles
        # .add(1).pow(1/lag).sub(1) normaliza los rendimientos a una base mensual
        df[f'return_{lag}m'] = (df['adj close']
                              .pct_change(lag)
                              .pipe(lambda x: x.clip(lower=x.quantile(outlier_cutoff),
                                                     upper=x.quantile(1-outlier_cutoff)))
                              .add(1)
                              .pow(1/lag)
                              .sub(1))
    return df

# Aplica la función calculate_returns a cada grupo de acciones (ticker)
# 'groupby(level=1, group_keys=False)' agrupa el DataFrame por ticker
# 'apply(calculate_returns)' aplica la función a cada grupo
# 'dropna()' elimina las filas con valores faltantes
data = data.groupby(level=1, group_keys=False).apply(calculate_returns).dropna()

# El DataFrame 'data' ahora incluye columnas adicionales para los rendimientos ajustados para cada uno de los lags definidos
data

adj close       atr   bb_high    bb_low    bb_mid  \
date       ticker                                                       
2017-10-31 AAL      45.534164  1.011062  3.994389  3.849110  3.921750   
           AAPL     39.818516 -0.906642  3.691040  3.597289  3.644164   
           ABBV     68.772308  0.375557  4.307973  4.215227  4.261600   
           ABT      48.969303 -1.040044  3.949284  3.902136  3.925710   
           ACN     130.375092 -0.986514  4.889487  4.810123  4.849805   
...                       ...       ...       ...       ...       ...   
2023-09-30 VRTX    351.690002  0.029799  5.879295  5.838959  5.859127   
           VZ       32.300549 -1.078816  3.563843  3.499366  3.531604   
           WFC      40.290813 -0.558742  3.790225  3.709473  3.749849   
           WMT     161.898697 -0.196379  5.113302  5.077929  5.095615   
           XOM     115.355156  0.601335  4.784477  4.704273  4.744375   

                   garman_klass_vol      macd        rsi  return_1m  \
date       ticker                                                     
2017-10-31 AAL            -0.000363 -0.018698  41.051779  -0.014108   
           AAPL           -0.000945 -0.039275  69.196732   0.096808   
           ABBV           -0.029822  0.473814  55.247882   0.022728   
           ABT            -0.004349  0.276132  53.844868   0.021276   
           ACN            -0.003359  0.352342  69.365147   0.064180   
...                             ...       ...        ...        ...   
2023-09-30 VRTX            0.000037  0.027907  52.406728   0.009617   
           VZ             -0.000067 -0.350385  42.222474  -0.056890   
           WFC             0.000136 -0.282325  40.920284  -0.015500   
           WMT             0.000011  0.399459  54.722511  -0.000676   
           XOM             0.000068  1.400623  59.440184   0.046947   

                   return_2m  return_3m  return_6m  return_9m  return_12m  
date       ticker                                                          
2017-10-31 AAL      0.022981  -0.023860   0.016495   0.007008    0.012702  
           AAPL     0.015250   0.044955   0.028875   0.038941    0.035228  
           ABBV     0.098590   0.091379   0.056495   0.047273    0.044026  
           ABT      0.034308   0.034801   0.038672   0.031320    0.029294  
           ACN      0.048455   0.037203   0.028692   0.027398    0.018728  
...                      ...        ...        ...        ...         ...  
2023-09-30 VRTX    -0.000923  -0.000208   0.018495   0.022140    0.016337  
           VZ      -0.016122  -0.033458  -0.021495  -0.014100   -0.006158  
           WFC     -0.057917  -0.013554   0.016712   0.000702    0.003255  
           WMT      0.010014   0.012354   0.017574   0.016553    0.020256  
           XOM      0.046139   0.030496   0.012838   0.008747    0.027037  

[10337 rows x 14 columns]

<div><font color="deepgray">

*Este paso es crucial para captar la dinámica de las series temporales y entender mejor el comportamiento de las acciones a lo largo del tiempo.*</font></div>

<div><font color="deepgray">

## **1. Cálculo de Rentabilidades Mensuales**</font></div>

<div><font color="deepgray">

**Qué es:** Calculamos los rendimientos mensuales para diferentes horizontes temporales (lags) utilizando el método .pct_change(lag). Esto implica calcular el cambio porcentual en el precio de cierre ajustado (adj close) para varios periodos mensuales.</font></div>

<div><font color="deepgray">

**Por qué es importante:** Estos rendimientos reflejan cómo ha cambiado el precio de una acción a lo largo del tiempo y son fundamentales para identificar patrones de impulso o tendencia en el mercado. Los diferentes horizontes temporales te permiten captar la dinámica del mercado a corto, medio y largo plazo.</font></div>

<div><font color="deepgray">

## **2. Control de Valores Atípicos**</font></div>

<div><font color="deepgray">

**Qué es:** Aplicamos un recorte (clip) a los rendimientos para limitar el impacto de valores atípicos extremos, utilizando un umbral de corte basado en cuantiles.</font></div>

<div><font color="deepgray">

**Por qué es importante:** Esto ayuda a mitigar el efecto de movimientos de precios extremos o inusuales, asegurando que el análisis no este sesgado por eventos raros o anómalos.</font></div>

<div><font color="deepgray">

## **3. Normalización de Rendimientos**</font></div>

<div><font color="deepgray">

**Qué es:** Normalizamos los rendimientos para cada horizonte temporal dividiendo por el número de meses en el lag y restando 1. Esto convierte los rendimientos en una base mensual comparable.</font></div>

<div><font color="deepgray">

**Por qué es importante:** Esta normalización permite comparar rendimientos a través de diferentes horizontes temporales de manera justa y coherente.</font></div>

<div><font color="deepgray">

## **4. Filtrado y Limpieza de Datos**</font></div>

<div><font color="deepgray">

Agrupamos los datos por 'ticker' y aplicamos la función calculate_returns. Luego, eliminmos las filas con valores faltantes. Este paso asegura que los cálculos de rendimiento se realicen de manera consistente para cada acción y que el conjunto de datos final esté limpio y sea completo.</font></div>

<div><font color="deepgray">

## **Resultado Final**</font></div>

<div><font color="deepgray">

El resultado es un DataFrame que incluye no solo los indicadores técnicos previamente calculados, sino también una serie de columnas de rendimientos para diferentes horizontes temporales. Estos rendimientos proporcionan una visión valiosa sobre cómo ha actuado cada acción en el pasado reciente y pueden ser indicativos de su comportamiento futuro.</font></div>

<div><font color="deepgray">

## **Importancia de Estos Pasos**</font></div>

<div><font color="deepgray">

**Análisis de Tendencias:** Los rendimientos históricos son una herramienta clave para identificar tendencias y patrones en el precio de las acciones.</font></div>

<div><font color="deepgray">

**Base para Estrategias de Trading:** Estos cálculos son esenciales para desarrollar estrategias de trading basadas en el análisis técnico y el análisis cuantitativo.</font></div>

<div><font color="deepgray">

**Evaluación del Riesgo:** Comprender cómo han fluctuado los precios en el pasado ayuda a evaluar el riesgo y la volatilidad de las inversiones.
Este enfoque te permite preparar los datos para análisis más avanzados, como la modelización predictiva.</font></div>

# **5. Descargar Factores Fama-French y Calcular Betas de Factores Móviles**

- Introduciremos los datos Fama-French para estimar la exposición de los activos a factores de riesgo comunes mediante regresión lineal.

- Se ha demostrado empíricamente que los cinco factores de Fama-French, riesgo de mercado, tamaño, valor, rentabilidad de explotación e inversión, explican la rentabilidad de los activos y se utilizan habitualmente para evaluar el perfil de riesgo/rentabilidad de las carteras. De ahí que resulte natural incluir en los modelos las exposiciones pasadas a los factores como características financieras.

- Podemos acceder a los rendimientos históricos de los factores utilizando el pandas-datareader y estimar las exposiciones históricas utilizando la regresión lineal móvil RollingOLS.

In [ ]:
# Descarga los datos de los cinco factores de Fama-French desde el año 2010
factor_data = web.DataReader('F-F_Research_Data_5_Factors_2x3',
                             'famafrench',
                             start='2010')[0].drop('RF', axis=1)  # Elimina el factor 'RF' (tasa libre de riesgo)

# Convierte el índice a marcas de tiempo (timestamps)
factor_data.index = factor_data.index.to_timestamp()

# Reescala los datos a una frecuencia mensual, tomando el último valor de cada mes
factor_data = factor_data.resample('M').last().div(100)  # Divide por 100 para convertir a formato porcentual

# Renombra el índice para que sea 'date'
factor_data.index.name = 'date'

# Une los rendimientos mensuales del DataFrame 'data' con los factores de Fama-French
# 'data['return_1m']' contiene los rendimientos mensuales de las acciones
factor_data = factor_data.join(data['return_1m']).sort_index()

# El DataFrame 'factor_data' ahora incluye los cinco factores de Fama-French junto con los rendimientos mensuales de las acciones
factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1m
date       ticker                                                   
2017-10-31 AAL     0.0225 -0.0194  0.0020  0.0093 -0.0325  -0.014108
           AAPL    0.0225 -0.0194  0.0020  0.0093 -0.0325   0.096808
           ABBV    0.0225 -0.0194  0.0020  0.0093 -0.0325   0.022728
           ABT     0.0225 -0.0194  0.0020  0.0093 -0.0325   0.021276
           ACN     0.0225 -0.0194  0.0020  0.0093 -0.0325   0.064180
...                   ...     ...     ...     ...     ...        ...
2023-09-30 VRTX   -0.0524 -0.0180  0.0152  0.0186 -0.0083   0.009617
           VZ     -0.0524 -0.0180  0.0152  0.0186 -0.0083  -0.056890
           WFC    -0.0524 -0.0180  0.0152  0.0186 -0.0083  -0.015500
           WMT    -0.0524 -0.0180  0.0152  0.0186 -0.0083  -0.000676
           XOM    -0.0524 -0.0180  0.0152  0.0186 -0.0083   0.046947

[10337 rows x 6 columns]

In [ ]:
factor_data.xs('AAPL', level=1).head()

,Mkt-RF,SMB,HML,RMW,CMA,return_1m
date,,,,,,
2017-10-31,0.0225,-0.0194,0.0020,0.0093,-0.0325,0.096808
2017-11-30,0.0312,-0.0033,-0.0003,0.0316,-0.0005,0.020278
2017-12-31,0.0106,-0.0107,0.0006,0.0074,0.0169,-0.015246
2018-01-31,0.0557,-0.0318,-0.0129,-0.0076,-0.0096,-0.010636
2018-02-28,-0.0365,0.0032,-0.0104,0.0052,-0.0237,0.068185


In [ ]:
factor_data.xs('MSFT', level=1).head()

,Mkt-RF,SMB,HML,RMW,CMA,return_1m
date,,,,,,
2017-10-31,0.0225,-0.0194,0.0020,0.0093,-0.0325,0.116660
2017-11-30,0.0312,-0.0033,-0.0003,0.0316,-0.0005,0.016984
2017-12-31,0.0106,-0.0107,0.0006,0.0074,0.0169,0.016277
2018-01-31,0.0557,-0.0318,-0.0129,-0.0076,-0.0096,0.110708
2018-02-28,-0.0365,0.0032,-0.0104,0.0052,-0.0237,-0.008415


* Filtrar los valores con menos de 10 meses de datos.

In [ ]:
# Agrupar los datos de factor_data por 'ticker' y contar el número de observaciones para cada uno
observations = factor_data.groupby(level=1).size()

# Filtrar para mantener solo aquellos tickers que tienen al menos 10 observaciones
# Esto asegura que solo se incluyan acciones con suficientes datos para análisis
valid_stocks = observations[observations >= 10]

# Filtrar factor_data para incluir solo las filas cuyos tickers están en el índice de valid_stocks
# Esto garantiza que solo se utilicen los datos de acciones con suficientes observaciones
factor_data = factor_data[factor_data.index.get_level_values('ticker').isin(valid_stocks.index)]

# El DataFrame resultante, factor_data, ahora contiene solo las acciones con un número suficiente de observaciones
factor_data

Mkt-RF     SMB     HML     RMW     CMA  return_1m
date       ticker                                                   
2017-10-31 AAL     0.0225 -0.0194  0.0020  0.0093 -0.0325  -0.014108
           AAPL    0.0225 -0.0194  0.0020  0.0093 -0.0325   0.096808
           ABBV    0.0225 -0.0194  0.0020  0.0093 -0.0325   0.022728
           ABT     0.0225 -0.0194  0.0020  0.0093 -0.0325   0.021276
           ACN     0.0225 -0.0194  0.0020  0.0093 -0.0325   0.064180
...                   ...     ...     ...     ...     ...        ...
2023-09-30 VRTX   -0.0524 -0.0180  0.0152  0.0186 -0.0083   0.009617
           VZ     -0.0524 -0.0180  0.0152  0.0186 -0.0083  -0.056890
           WFC    -0.0524 -0.0180  0.0152  0.0186 -0.0083  -0.015500
           WMT    -0.0524 -0.0180  0.0152  0.0186 -0.0083  -0.000676
           XOM    -0.0524 -0.0180  0.0152  0.0186 -0.0083   0.046947

[10297 rows x 6 columns]

* Calcular Rolling Factor Betas.

In [ ]:
# Calcular las betas de los factores para cada acción utilizando una regresión lineal móvil
betas = (factor_data.groupby(level=1,  # Agrupar por 'ticker'
                            group_keys=False)  # No incluir las claves de grupo en el resultado
         .apply(lambda x: RollingOLS(endog=x['return_1m'],  # Variable dependiente: rendimiento mensual
                                     exog=sm.add_constant(x.drop('return_1m', axis=1)),  # Variables independientes: factores Fama-French con una constante añadida
                                     window=min(24, x.shape[0]),  # Tamaño de la ventana para la regresión móvil, mínimo entre 24 meses y el número total de observaciones
                                     min_nobs=len(x.columns)+1)  # Número mínimo de observaciones para realizar la regresión
         .fit(params_only=True)  # Ajustar el modelo y obtener solo los parámetros
         .params  # Extraer los parámetros (betas) del modelo ajustado
         .drop('const', axis=1)))  # Eliminar la constante del modelo

# El DataFrame 'betas' contiene las betas de los factores para cada acción
betas


Mkt-RF       SMB       HML       RMW       CMA
date       ticker                                                  
2017-10-31 AAL          NaN       NaN       NaN       NaN       NaN
           AAPL         NaN       NaN       NaN       NaN       NaN
           ABBV         NaN       NaN       NaN       NaN       NaN
           ABT          NaN       NaN       NaN       NaN       NaN
           ACN          NaN       NaN       NaN       NaN       NaN
...                     ...       ...       ...       ...       ...
2023-09-30 VRTX    0.456467 -0.445783 -0.312765 -0.079230  0.800476
           VZ      0.332655 -0.165646  0.267344  0.310366  0.106902
           WFC     1.121747  0.300521  2.061532 -0.439488 -1.517558
           WMT     0.700556 -0.315599 -0.413432 -0.142907  0.508471
           XOM     0.983510 -1.093757  1.756776 -0.640866 -0.368454

[10297 rows x 5 columns]

* Unir los datos de los factores móviles al marco de datos de las características principales.

In [ ]:
# Definir los nombres de los factores Fama-French
factors = ['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']

# Unir los datos de betas al DataFrame principal 'data'
data = (data.join(betas.groupby('ticker').shift()))  # Desplazar las betas para evitar mirar hacia adelante

# Rellenar los valores faltantes de los factores para cada acción
# Utilizamos la media de cada factor para cada acción como valor de relleno
data.loc[:, factors] = data.groupby('ticker', group_keys=False)[factors].apply(lambda x: x.fillna(x.mean()))

# Eliminar la columna 'adj close' ya que no se necesita para el análisis posterior
data = data.drop('adj close', axis=1)

# Eliminar cualquier fila que tenga valores faltantes
data = data.dropna()

# Mostrar información sobre el DataFrame resultante
data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 10058 entries, (Timestamp('2017-10-31 00:00:00', freq='M'), 'AAL') to (Timestamp('2023-09-30 00:00:00', freq='M'), 'XOM')
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   atr               10058 non-null  float64
 1   bb_high           10058 non-null  float64
 2   bb_low            10058 non-null  float64
 3   bb_mid            10058 non-null  float64
 4   garman_klass_vol  10058 non-null  float64
 5   macd              10058 non-null  float64
 6   rsi               10058 non-null  float64
 7   return_1m         10058 non-null  float64
 8   return_2m         10058 non-null  float64
 9   return_3m         10058 non-null  float64
 10  return_6m         10058 non-null  float64
 11  return_9m         10058 non-null  float64
 12  return_12m        10058 non-null  float64
 13  Mkt-RF            10058 non-null  float64
 14  SMB               10058 non-null  floa

<div><font color="deepgray">

*Estas implementaciones son esenciales para comprender la exposición de los activos a factores de riesgo comunes y para evaluar el perfil de riesgo/rentabilidad de las carteras.*</font></div>

<div><font color="deepgray">

## **1. Integración de Factores Fama-French**</font></div>

<div><font color="deepgray">

**Qué es:** Los factores Fama-French son un conjunto de factores de riesgo comunes en los rendimientos de las acciones. Incluyen el riesgo de mercado (Mkt-RF), tamaño (SMB), valor (HML), rentabilidad de explotación (RMW) e inversión (CMA).</font></div>

<div><font color="deepgray">

**Por qué es importante:** Estos factores ayudan a explicar por qué diferentes acciones tienen diferentes rendimientos y proporcionan una comprensión más profunda de las fuerzas que impulsan los rendimientos de las acciones y su relación con el mercado en general.</font></div>

<div><font color="deepgray">

## **2. Cálculo de Betas de Factores Móviles**</font></div>

<div><font color="deepgray">

**Qué es:** Las betas de factores son coeficientes que miden la sensibilidad de los rendimientos de una acción a los cambios en los factores de riesgo. Se estiman a lo largo del tiempo usando un modelo de regresión lineal móvil (RollingOLS).</font></div>

<div><font color="deepgray">

**Por qué es importante:** Las betas de factores móviles permiten ver cómo la sensibilidad de una acción a los factores de riesgo cambia con el tiempo, lo cual es crucial para entender cómo el perfil de riesgo de una acción evoluciona y cómo podría comportarse bajo diferentes condiciones de mercado.</font></div>

<div><font color="deepgray">

## **3. Unión de Datos de Factores y Características Principales**</font></div>

<div><font color="deepgray">

**Qué es:** Combina los datos de betas de factores con las características principales (como volatilidad, RSI, MACD, etc.) en un solo DataFrame.</font></div>

<div><font color="deepgray">

**Por qué es importante:** Al unir estos datos, se crea un conjunto de datos completo que captura tanto las características internas de las acciones (como la volatilidad y el momento) como su relación con factores de riesgo externos.</font></div>

<div><font color="deepgray">

## **4. Limpieza y Preparación de Datos**</font></div>

<div><font color="deepgray">

Eliminamos las columnas innecesarias y manejamos los valores faltantes para asegurar de que el conjunto de datos esté limpio y listo para el análisi esebcial para realizar un análisis preciso y confiable, asegurando que las conclusiones que saquemos de los datos sean válidas y aplicables.</font></div>

<div><font color="deepgray">

## **Resultado Final**</font></div>

<div><font color="deepgray">

En resumen, en este paso se ha enriquecido el conjunto de datos con información importante sobre cómo cada acción se relaciona con factores de riesgo comunes en el mercado.</font></div>

<div><font color="deepgray">

## **Importancia de Estos Pasos**</font></div>

<div><font color="deepgray">

**Análisis:** Esto no solo mejora la comprensión de cada acción individual, sino que también te prepara para realizar análisis más avanzados, como la construcción de carteras y la optimización de la relación riesgo-rendimiento.</font></div>

# En este punto tenemos que decidir qué modelo ML y qué enfoque utilizar para las predicciones, etc.

# **6. Para cada mes, se aplica un algoritmo de agrupación de K-Means para agrupar activos similares en función de sus características**

## K-Means Clustering
* Es posible que deseemos inicializar centroides predefinidos para cada conglomerado basándonos en la investigación.

* Para fines de visualización nos basaremos inicialmente en la inicialización 'k-means++'.

* A continuación, predefiniremos nuestros centroides para cada conglomerado.

In [ ]:
# Definir valores objetivo para el indicador RSI (Relative Strength Index)
target_rsi_values = [30, 45, 55, 70]

# Inicializar una matriz de ceros para los centroides
# La matriz tiene una fila para cada valor RSI objetivo y 18 columnas (características)
initial_centroids = np.zeros((len(target_rsi_values), 18))

# Asignar los valores objetivo de RSI a la séptima columna (índice 6) de la matriz de centroides
# Esto establece los valores de RSI objetivo como los centroides iniciales para el clustering
initial_centroids[:, 6] = target_rsi_values

# La matriz 'initial_centroids' ahora contiene centroides predefinidos
# donde cada centroide tiene un valor específico de RSI y ceros en las demás características
initial_centroids

array([[ 0.,  0.,  0.,  0.,  0.,  0., 30.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., 45.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., 55.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0., 70.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.]])

In [ ]:
from sklearn.cluster import KMeans

# data = data.drop('cluster', axis=1)

# Función para aplicar el algoritmo de K-Means y obtener clusters
def get_clusters(df):
    # Aplicar K-Means para agrupar los datos en 4 clusters
    # 'random_state=0' asegura la reproducibilidad
    # 'init=initial_centroids' utiliza centroides predefinidos para la inicialización
    df['cluster'] = KMeans(n_clusters=4,
                           random_state=0,
                           init=initial_centroids).fit(df).labels_
    return df

# Eliminar filas con valores faltantes antes de aplicar K-Means
data = data.dropna()

# Aplicar la función 'get_clusters' a cada grupo de datos por fecha
# Esto agrupa los activos en clusters para cada fecha específica
data = data.groupby('date', group_keys=False).apply(get_clusters)

# El DataFrame 'data' ahora incluye una nueva columna 'cluster'
# que indica a qué cluster pertenece cada activo en cada fecha

data

atr   bb_high    bb_low    bb_mid  garman_klass_vol  \
date       ticker                                                             
2017-10-31 AAL     1.011062  3.994389  3.849110  3.921750         -0.000363   
           AAPL   -0.906642  3.691040  3.597289  3.644164         -0.000945   
           ABBV    0.375557  4.307973  4.215227  4.261600         -0.029822   
           ABT    -1.040044  3.949284  3.902136  3.925710         -0.004349   
           ACN    -0.986514  4.889487  4.810123  4.849805         -0.003359   
...                     ...       ...       ...       ...               ...   
2023-09-30 VRTX    0.029799  5.879295  5.838959  5.859127          0.000037   
           VZ     -1.078816  3.563843  3.499366  3.531604         -0.000067   
           WFC    -0.558742  3.790225  3.709473  3.749849          0.000136   
           WMT    -0.196379  5.113302  5.077929  5.095615          0.000011   
           XOM     0.601335  4.784477  4.704273  4.744375          0.000068   

                       macd        rsi  return_1m  return_2m  return_3m  \
date       ticker                                                         
2017-10-31 AAL    -0.018698  41.051779  -0.014108   0.022981  -0.023860   
           AAPL   -0.039275  69.196732   0.096808   0.015250   0.044955   
           ABBV    0.473814  55.247882   0.022728   0.098590   0.091379   
           ABT     0.276132  53.844868   0.021276   0.034308   0.034801   
           ACN     0.352342  69.365147   0.064180   0.048455   0.037203   
...                     ...        ...        ...        ...        ...   
2023-09-30 VRTX    0.027907  52.406728   0.009617  -0.000923  -0.000208   
           VZ     -0.350385  42.222474  -0.056890  -0.016122  -0.033458   
           WFC    -0.282325  40.920284  -0.015500  -0.057917  -0.013554   
           WMT     0.399459  54.722511  -0.000676   0.010014   0.012354   
           XOM     1.400623  59.440184   0.046947   0.046139   0.030496   

                   return_6m  return_9m  return_12m    Mkt-RF       SMB  \
date       ticker                                                         
2017-10-31 AAL      0.016495   0.007008    0.012702  1.265664  1.314316   
           AAPL     0.028875   0.038941    0.035228  1.275749 -0.271696   
           ABBV     0.056495   0.047273    0.044026  0.494521  0.362282   
           ABT      0.038672   0.031320    0.029294  0.828092 -0.199021   
           ACN      0.028692   0.027398    0.018728  1.199048 -0.166701   
...                      ...        ...         ...       ...       ...   
2023-09-30 VRTX     0.018495   0.022140    0.016337  0.504915 -0.411099   
           VZ      -0.021495  -0.014100   -0.006158  0.302531 -0.212073   
           WFC      0.016712   0.000702    0.003255  1.137640  0.314482   
           WMT      0.017574   0.016553    0.020256  0.740055 -0.261889   
           XOM      0.012838   0.008747    0.027037  1.005967 -1.051368   

                        HML       RMW       CMA  cluster  
date       ticker                                         
2017-10-31 AAL     0.601340  0.458859  0.563294        1  
           AAPL   -0.591144  0.647780  0.458079        3  
           ABBV   -0.033101  0.241984  0.164920        2  
           ABT    -0.529385  0.252406  0.970212        2  
           ACN    -0.328215  0.267788  0.168884        3  
...                     ...       ...       ...      ...  
2023-09-30 VRTX   -0.481425  0.057280  0.943566        3  
           VZ      0.344589  0.318168  0.000826        2  
           WFC     2.009074 -0.404336 -1.468840        1  
           WMT    -0.522676 -0.126426  0.642627        3  
           XOM     1.707833 -0.675679 -0.284025        3  

[10058 rows x 19 columns]

In [ ]:
def plot_clusters(data):
    # Filtrar los datos para obtener solo aquellos pertenecientes al cluster 0
    cluster_0 = data[data['cluster'] == 0]
    # Filtrar los datos para obtener solo aquellos pertenecientes al cluster 1
    cluster_1 = data[data['cluster'] == 1]
    # Filtrar los datos para obtener solo aquellos pertenecientes al cluster 2
    cluster_2 = data[data['cluster'] == 2]
    # Filtrar los datos para obtener solo aquellos pertenecientes al cluster 3
    cluster_3 = data[data['cluster'] == 3]

    # Crear un gráfico de dispersión para el cluster 0
    plt.scatter(cluster_0.iloc[:, 0], cluster_0.iloc[:, 6], color='red', label='cluster 0')
    # Crear un gráfico de dispersión para el cluster 1
    plt.scatter(cluster_1.iloc[:, 0], cluster_1.iloc[:, 6], color='green', label='cluster 1')
    # Crear un gráfico de dispersión para el cluster 2
    plt.scatter(cluster_2.iloc[:, 0], cluster_2.iloc[:, 6], color='blue', label='cluster 2')
    # Crear un gráfico de dispersión para el cluster 3
    plt.scatter(cluster_3.iloc[:, 0], cluster_3.iloc[:, 6], color='black', label='cluster 3')

    # Añadir una leyenda al gráfico
    plt.legend()
    # Mostrar el gráfico
    plt.show()
    return

In [ ]:
import plotly.graph_objects as go

# Función para visualizar los clusters para un conjunto de datos específico
def plot_clusters_plotly(data, date):
    fig = go.Figure()

    for cluster_id, color in zip(range(4), ['red', 'green', 'blue', 'black']):
        cluster_data = data[data['cluster'] == cluster_id]
        fig.add_trace(go.Scatter(x=cluster_data.iloc[:, 0], y=cluster_data.iloc[:, 6],
                                 mode='markers',
                                 marker=dict(color=color),
                                 name=f'Cluster {cluster_id}'))

    fig.update_layout(title=f'Clusters for Date: {date}',
                      xaxis_title=data.columns[0],
                      yaxis_title=data.columns[6],
                      legend_title="Cluster ID")

    fig.show()

# Iterar sobre cada fecha única en el nivel 'date' del índice del DataFrame 'data'
for i in data.index.get_level_values('date').unique().tolist():
    # Extraer los datos correspondientes a la fecha actual 'i'
    g = data.xs(i, level=0)
    # Llamar a la función plot_clusters_plotly para visualizar los clusters para la fecha 'i'
    plot_clusters_plotly(g, i)

<div><font color="deepgray">

*Aplicación del Algoritmo de Agrupación K-Means para Agrupar Activos Similares*</font></div>

<div><font color="deepgray">

## **1. K-Means Clustering**</font></div>

<div><font color="deepgray">

**Qué es:** K-Means es un algoritmo de aprendizaje no supervisado que agrupa datos en función de sus características. Agrupa activos en 'k' número de conglomerados basándose en similitudes en sus características.</font></div>

<div><font color="deepgray">

**Por qué es importante:** Este enfoque permite identificar grupos de activos con comportamientos similares, lo que puede ser útil para diversificar una cartera, identificar oportunidades de inversión o desarrollar estrategias de trading específicas para cada grupo.</font></div>

<div><font color="deepgray">

## **2. Inicialización y Predefinición de Centroides**</font></div>

<div><font color="deepgray">

**Qué es:** La inicialización de centroides es un paso crucial en K-Means. Seleccionamos 'k-means++' para una inicialización eficiente y también predefinimos nuestros centroides basándonos en la investigación y objetivos específicos.</font></div>

<div><font color="deepgray">

**Por qué es importante:** Una buena inicialización de centroides puede llevar a una convergencia más rápida y a resultados más precisos en la agrupación. La predefinición de centroides permite incorporar conocimientos previos o hipótesis específicas en el modelo.</font></div>

<div><font color="deepgray">

## **3. Aplicación del Modelo y Visualización**</font></div>

<div><font color="deepgray">

**Qué es:** Aplicamos el modelo K-Means a nuestros datos y visualizamos los resultados para entender cómo se distribuyen los activos en los diferentes conglomerados.</font></div>

<div><font color="deepgray">

**Por qué es importante:** La visualización ayuda a interpretar los resultados del modelo, permitiendo identificar patrones y diferencias clave entre los conglomerados. Esto puede proporcionar insights valiosos para la toma de decisiones de inversión.</font></div>

<div><font color="deepgray">

## **4. Resultados y Análisis**</font></div>

<div><font color="deepgray">

**Qué es:** Observamos los resultados de la agrupación y analizamos las características comunes dentro de cada conglomerado, así como las diferencias entre ellos.</font></div>

<div><font color="deepgray">

**Por qué es importante:** Este análisis proporciona una comprensión profunda de las características que definen cada grupo de activos y puede ser crucial para desarrollar estrategias de inversión basadas en el perfil de riesgo, rendimiento esperado o características específicas del mercado.</font></div>

<div><font color="deepgray">

## **Importancia de Estos Pasos**</font></div>

<div><font color="deepgray">

**Segmentación de Activos:** La agrupación de activos ayuda a segmentar el mercado en grupos manejables, facilitando la toma de decisiones de inversión más informadas.</font></div>

<div><font color="deepgray">

**Desarrollo de Estrategias:** Los resultados de K-Means pueden ser utilizados para desarrollar estrategias de trading específicas para cada conglomerado, aprovechando sus características únicas.</font></div>

<div><font color="deepgray">

**Análisis de Diversificación:** La comprensión de cómo se agrupan los activos puede ser útil para la construcción de carteras diversificadas y la gestión del riesgo.</font></div>

<div><font color="deepgray">

##**¿Qué Estamos Visualizando?**</font></div>

<div><font color="deepgray">

Al aplicar K-Means y visualizar los clusters, estamos observando cómo el algoritmo ha agrupado los activos en función de sus características (como volatilidad, rendimientos históricos, etc). Cada cluster representará un grupo de acciones con características similares. Esto es lo que estamos visualizando:</font></div>

<div><font color="deepgray">

**Distribución de los Activos:** Cómo se distribuyen los activos en diferentes grupos basados en sus características financieras.</font></div>

<div><font color="deepgray">

**Relaciones entre Características:** Cómo ciertas características (como el RSI o la volatilidad) influyen en la formación de estos grupos.</font></div>

<div><font color="deepgray">

**Identificación de Patrones:** Patrones comunes dentro de cada grupo que podrían no ser evidentes sin un análisis de cluster.</font></div>

<div><font color="deepgray">

Por ejemplo, si se observa que ciertas acciones siempre se agrupan juntas, podríamos inferir que comparten ciertas dinámicas de mercado. Esto podría ser crucial para la toma de decisiones en inversión o en la construcción de una cartera de acciones. La visualización de estos clusters te ayuda a comprender mejor la estructura subyacente de los datos y a tomar decisiones más informadas basadas en el análisis de estos grupos.</font></div>

# **7. Para cada mes, seleccionamos los activos en función de la agrupación y armamos una cartera basada en la optimización del Efficient Frontier Max Sharpe Ratio**

* En primer lugar, filtraremos sólo las acciones correspondientes al clúster que elijamos en función de nuestra hipótesis.

* El momentum es persistente en torno al centroide RSI 70 y mi idea sería que los valores agrupados deberían seguir obteniendo mejores resultados en el mes siguiente, por lo que seleccionaría los valores correspondientes al grupo 3.


In [ ]:
# Filtrar los datos para seleccionar solo aquellos que pertenecen al cluster 3
filtered_df = data[data['cluster'] == 3].copy()

# Restablecer el índice para el nivel 'ticker', manteniendo 'date' como índice
filtered_df = filtered_df.reset_index(level=1)

# Ajustar el índice 'date' sumando un día a cada fecha
# Esto se hace para alinear los datos con el mes siguiente
filtered_df.index = filtered_df.index + pd.DateOffset(1)

# Restablecer el índice y luego establecer ['date', 'ticker'] como índices multinivel
filtered_df = filtered_df.reset_index().set_index(['date', 'ticker'])

# Obtener una lista de fechas únicas en el índice 'date'
dates = filtered_df.index.get_level_values('date').unique().tolist()

# Inicializar un diccionario para almacenar las fechas fijas
fixed_dates = {}

# Iterar sobre cada fecha única
for d in dates:
    # Convertir la fecha a formato de cadena y almacenar los tickers correspondientes
    # a esa fecha en el diccionario
    fixed_dates[d.strftime('%Y-%m-%d')] = filtered_df.xs(d, level=0).index.tolist()

# Mostrar el diccionario de fechas fijas
fixed_dates

{'2017-11-01': ['AAPL',
  'ACN',
  'ADBE',
  'AMAT',
  'AMZN',
  'AVGO',
  'AXP',
  'BAC',
  'CAT',
  'COP',
  'CRM',
  'CTSH',
  'DE',
  'DHR',
  'DLTR',
  'ELV',
  'EOG',
  'GOOG',
  'GOOGL',
  'INTC',
  'ISRG',
  'JPM',
  'MA',
  'MAR',
  'MCD',
  'MET',
  'META',
  'MMM',
  'MSFT',
  'MU',
  'NEE',
  'NVDA',
  'NXPI',
  'ORCL',
  'PYPL',
  'SHW',
  'STZ',
  'TXN',
  'UNH',
  'V',
  'VLO',
  'WMT',
  'XOM'],
 '2017-12-01': ['AMZN',
  'AXP',
  'AZO',
  'BA',
  'BAC',
  'BRK-B',
  'CAT',
  'COST',
  'CSCO',
  'DE',
  'DLTR',
  'ELV',
  'HD',
  'HON',
  'ILMN',
  'JPM',
  'KR',
  'LUV',
  'MAR',
  'MMM',
  'NKE',
  'ORLY',
  'OXY',
  'SCHW',
  'UNH',
  'UNP',
  'VLO',
  'VZ',
  'WMT'],
 '2018-01-01': ['ACN',
  'BA',
  'BAC',
  'CAT',
  'CSCO',
  'CVX',
  'DAL',
  'DE',
  'DG',
  'EOG',
  'FCX',
  'FDX',
  'HAL',
  'HD',
  'KR',
  'LOW',
  'LUV',
  'MAR',
  'OXY',
  'PEP',
  'PXD',
  'RTX',
  'STZ',
  'T',
  'TXN',
  'UAL',
  'UNP',
  'VLO',
  'VZ',
  'WMT',
  'WYNN'],
 '2018-02-01': ['

## **Definición la función de optimización del portafolio**

* Definiremos una función que optimice los weights de la cartera utilizando el paquete PyPortfolioOpt y el optimizador EfficientFrontier para maximizar el ratio de sharpe.

* Para optimizar los weights de una cartera determinada, tendríamos que suministrar a la función los precios del último año.

* Aplicar la restricción de los límites de los weights de las acciones para la diversificación (mínimo de la mitad del mismo peso y máximo del 10% del portafolio).

In [ ]:
# Importar las bibliotecas necesarias para la optimización de carteras
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns

# Definir una función para optimizar los pesos de una cartera
def optimize_weights(prices, lower_bound=0):
    # Calcular los rendimientos esperados utilizando el retorno histórico medio
    returns = expected_returns.mean_historical_return(prices=prices,
                                                      frequency=252)  # 252 días de trading en un año

    # Calcular la matriz de covarianza de los rendimientos de las acciones
    cov = risk_models.sample_cov(prices=prices,
                                 frequency=252)  # 252 días de trading en un año

    # Inicializar el objeto EfficientFrontier con los rendimientos esperados y la matriz de covarianza
    ef = EfficientFrontier(expected_returns=returns,
                           cov_matrix=cov,
                           weight_bounds=(lower_bound, .1),  # Establecer límites para los pesos de las acciones
                           solver='SCS')  # Usar el solver 'SCS'

    # Encontrar los pesos de la cartera que maximizan el ratio de Sharpe
    weights = ef.max_sharpe()

    # Limpiar los pesos para una mejor presentación
    return ef.clean_weights()

* Descargar datos de precios diarios sólo para valores cotizados en corto.

In [ ]:
# Obtener una lista de todos los tickers únicos presentes en el DataFrame 'data'
stocks = data.index.get_level_values('ticker').unique().tolist()

# Descargar datos de precios diarios de Yahoo Finance para los tickers en la lista
new_df = yf.download(tickers=stocks,
                     # Establecer la fecha de inicio para la descarga: un año antes de la primera fecha en 'data'
                     start=data.index.get_level_values('date').unique()[0]-pd.DateOffset(months=12),
                     # Establecer la fecha de finalización para la descarga: la última fecha en 'data'
                     end=data.index.get_level_values('date').unique()[-1])

# 'new_df' contendrá los precios diarios ajustados al cierre (y otros datos) para cada acción en 'stocks'
new_df

[*********************100%%**********************]  154 of 154 completed


Adj Close                                                 \
                  AAL        AAPL        ABBV        ABT         ACN   
Date                                                                   
2016-10-31  39.134335   26.281523   41.009071  34.630028  104.350311   
2016-11-01  38.363216   25.807001   41.501667  34.462341  104.260544   
2016-11-02  38.276466   25.830143   41.751629  34.153465  106.477898   
2016-11-03  38.054203   25.553284   41.089958  33.871063  104.987686   
2016-11-04  38.402084   25.322941   41.200233  34.497639  105.239067   
...               ...         ...         ...        ...         ...   
2023-09-25  12.910000  175.848328  153.117371  96.933411  315.680695   
2023-09-26  12.700000  171.733749  152.384705  95.700241  309.327057   
2023-09-27  12.610000  170.205750  151.612442  95.004089  313.081512   
2023-09-28  12.920000  170.465424  150.741165  97.579834  299.527710   
2023-09-29  12.810000  170.984741  147.582764  96.316818  305.841522   

                                                                       ...  \
                  ADBE         ADP        ADSK        AIG        AMAT  ...   
Date                                                                   ...   
2016-10-31  107.510002   74.686073   72.279999  50.977715   26.700010  ...   
2016-11-01  106.870003   74.565956   70.099998  50.134968   26.534740  ...   
2016-11-02  105.889999   77.191048   68.680000  50.027550   26.250109  ...   
2016-11-03  107.169998   76.633430   67.610001  48.044643   25.956297  ...   
2016-11-04  106.199997   76.418976   69.440002  47.408443   25.910393  ...   
...                ...         ...         ...        ...         ...  ...   
2023-09-25  511.600006  238.573792  205.669998  61.795918  136.297150  ...   
2023-09-26  506.299988  235.810547  201.660004  60.930592  133.792526  ...   
2023-09-27  502.600006  241.168076  202.279999  60.681938  134.770416  ...   
2023-09-28  504.670013  241.843964  207.889999  60.950485  137.923645  ...   
2023-09-29  509.899994  239.130417  206.910004  60.274143  138.153152  ...   

              Volume                                                           \
                   V      VLO     VRTX        VZ       WBA      WDC       WFC   
Date                                                                            
2016-10-31  10024000  4969500  1928200  12459400   4662100  3363200  20115900   
2016-11-01  10881500  7816800  2458200  13229400   5130800  3821100  20020200   
2016-11-02   9170900  7317600  2580400  16488200   9783100  4216200  19566600   
2016-11-03   7563100  3855900  2371000  12605100   4586500  3192600  14982700   
2016-11-04   7588100  3729900  1902100  14410200   7023900  2705600  27391600   
...              ...      ...      ...       ...       ...      ...       ...   
2023-09-25   5921600  3241900   698100  17616900   6891000  2685900  10624000   
2023-09-26   6193200  4936800   633600  18841600   7570500  3571100  15219400   
2023-09-27   6006700  3644000   860600  22083500  10182900  2547800  11815500   
2023-09-28   4203900  3587300   578900  18772100   7291500  3282000  12454600   
2023-09-29   6044200  4302200   896800  19759400  25659100  2512000  13119100   

                                        
                WMT     WYNN       XOM  
Date                                    
2016-10-31  6389000  1341600  16663800  
2016-11-01  8838600  3722300  13050600  
2016-11-02  7645300  2542400  11226100  
2016-11-03  6803900  9050400   8836500  
2016-11-04  7262100  3223800  13877100  
...             ...      ...       ...  
2023-09-25  3486500  1387800  11316000  
2023-09-26  4811900  1454000  11805400  
2023-09-27  5237000  2050000  23976200  
2023-09-28  3872400  1290400  16808100  
2023-09-29  6279300  1326800  18810700  

[1740 rows x 924 columns]

* Calcular la rentabilidad diaria de cada valor que podría entrar en nuestra cartera.

* Luego, realizar un bucle sobre cada inicio de mes, seleccionar las acciones del mes y calcular sus weights para el mes siguiente.

* Si la optimización del ratio sharpe máximo falla para un mes determinado, aplicar ponderaciones equitativas (equally-weighted weights).

* Calcular la rentabilidad diaria de la cartera.

In [ ]:
# Calcular la rentabilidad diaria de cada acción en el DataFrame 'new_df'
returns_dataframe = np.log(new_df['Adj Close']).diff()

# Inicializar un DataFrame vacío para almacenar los retornos de la cartera
portfolio_df = pd.DataFrame()

# Iterar sobre cada fecha de inicio en el diccionario 'fixed_dates'
for start_date in fixed_dates.keys():
    try:
        # Calcular la fecha de finalización del mes para la fecha de inicio actual
        end_date = (pd.to_datetime(start_date)+pd.offsets.MonthEnd(0)).strftime('%Y-%m-%d')

        # Obtener los tickers de las acciones para la fecha de inicio actual
        cols = fixed_dates[start_date]

        # Calcular las fechas de inicio y finalización para la optimización de la cartera
        optimization_start_date = (pd.to_datetime(start_date)-pd.DateOffset(months=12)).strftime('%Y-%m-%d')
        optimization_end_date = (pd.to_datetime(start_date)-pd.DateOffset(days=1)).strftime('%Y-%m-%d')

        # Obtener los precios de cierre ajustados para el período de optimización
        optimization_df = new_df[optimization_start_date:optimization_end_date]['Adj Close'][cols]

        # Inicializar una variable para rastrear el éxito de la optimización
        success = False
        try:
            # Intentar optimizar los pesos de la cartera para maximizar el ratio de Sharpe
            weights = optimize_weights(prices=optimization_df, lower_bound=round(1/(len(optimization_df.columns)*2),3))
            weights = pd.DataFrame(weights, index=pd.Series(0))
            success = True
        except:
            # Si falla la optimización, imprimir un mensaje y continuar con pesos iguales
            print(f'Max Sharpe Optimization failed for {start_date}, Continuing with Equal-Weights')

        # Si la optimización falló, establecer pesos iguales para todas las acciones
        if success==False:
            weights = pd.DataFrame([1/len(optimization_df.columns) for i in range(len(optimization_df.columns))],
                                   index=optimization_df.columns.tolist(),
                                   columns=pd.Series(0)).T

        # Calcular la rentabilidad diaria de la cartera para el período seleccionado
        temp_df = returns_dataframe[start_date:end_date]
        temp_df = temp_df.stack().to_frame('return').reset_index(level=0)\
                   .merge(weights.stack().to_frame('weight').reset_index(level=0, drop=True),
                          left_index=True,
                          right_index=True)\
                   .reset_index().set_index(['Date', 'index']).unstack().stack()
        temp_df.index.names = ['date', 'ticker']
        temp_df['weighted_return'] = temp_df['return']*temp_df['weight']
        temp_df = temp_df.groupby(level=0)['weighted_return'].sum().to_frame('Strategy Return')

        # Añadir los retornos de la cartera al DataFrame 'portfolio_df'
        portfolio_df = pd.concat([portfolio_df, temp_df], axis=0)

    except Exception as e:
        # Imprimir cualquier error que ocurra durante el proceso
        print(e)

# Eliminar duplicados en 'portfolio_df'
portfolio_df = portfolio_df.drop_duplicates()

# 'portfolio_df' contiene ahora los retornos diarios de la estrategia de cartera
portfolio_df

Max Sharpe Optimization failed for 2018-04-01, Continuing with Equal-Weights
Max Sharpe Optimization failed for 2018-05-01, Continuing with Equal-Weights
Max Sharpe Optimization failed for 2020-03-01, Continuing with Equal-Weights
Max Sharpe Optimization failed for 2020-04-01, Continuing with Equal-Weights
Max Sharpe Optimization failed for 2021-02-01, Continuing with Equal-Weights
Max Sharpe Optimization failed for 2021-10-01, Continuing with Equal-Weights
Max Sharpe Optimization failed for 2022-09-01, Continuing with Equal-Weights
Max Sharpe Optimization failed for 2022-10-01, Continuing with Equal-Weights
'return'


,Strategy Return
date,
2017-11-01,0.001361
2017-11-02,0.002909
2017-11-03,0.006310
2017-11-06,0.003008
2017-11-07,0.002704
...,...
2023-09-25,0.003641
2023-09-26,-0.011470
2023-09-27,0.005189


<div><font color="deepgray">

*Selección de Activos y Formación de Cartera Basada en la Optimización del Efficient Frontier Max Sharpe Ratio*</font></div>

<div><font color="deepgray">

## **1. Filtrado de Acciones por Clúster**</font></div>

<div><font color="deepgray">

**Qué es:** Seleccionamos acciones del clúster elegido (en este caso, el clúster 3) que, según nuestra hipótesis, deberían seguir obteniendo buenos resultados. Este enfoque se basa en la persistencia del momentum alrededor del centroide RSI 70.</font></div>

<div><font color="deepgray">

**Por qué es importante:** Este método permite identificar un grupo específico de acciones que tienen características deseables, como un alto momentum, lo que puede ser crucial para estrategias de inversión a corto plazo.</font></div>

<div><font color="deepgray">

## **2. Optimización de Pesos de la Cartera**</font></div>

<div><font color="deepgray">

**Qué es:** Utilizamos PyPortfolioOpt y el optimizador EfficientFrontier para maximizar el ratio de Sharpe. Aplicamos restricciones a los pesos de las acciones para asegurar la diversificación.</font></div>

<div><font color="deepgray">

**Por qué es importante:** La optimización del ratio de Sharpe busca maximizar la rentabilidad ajustada al riesgo, lo que es crucial para una gestión eficiente de la cartera.</font></div>

<div><font color="deepgray">

## **3. Descarga de Datos de Precios Diarios y Cálculo de Rentabilidad Diaria**</font></div>

<div><font color="deepgray">

**Qué es:** Obtenemos datos de precios diarios para las acciones seleccionadas y calculamos la rentabilidad diaria de cada valor que podría entrar en nuestra cartera.</font></div>

<div><font color="deepgray">

**Por qué es importante:** Estos cálculos son esenciales para entender el comportamiento diario de las acciones seleccionadas y para calcular la rentabilidad diaria de la cartera optimizada.</font></div>

<div><font color="deepgray">

## **4. Visualización de la Rentabilidad de la Cartera**</font></div>

<div><font color="deepgray">

**Qué es:** Visualizamos la rentabilidad diaria de la cartera optimizada, comparándola con los benchmarks relevantes.</font></div>

<div><font color="deepgray">

**Por qué es importante:** Esta visualización ayuda a evaluar el rendimiento de la cartera en comparación con el mercado en general y es crucial para la toma de decisiones de inversión.</font></div>

<div><font color="deepgray">

## **Importancia de Este Paso**</font></div>

<div><font color="deepgray">

**Estrategia Basada en Datos:** La selección de activos y la formación de la cartera se basan en un análisis cuantitativo riguroso, lo que aumenta la probabilidad de éxito de la estrategia de trading.</font></div>

<div><font color="deepgray">

**Optimización del Riesgo-Rentabilidad:** La optimización del ratio de Sharpe busca maximizar la rentabilidad ajustada al riesgo, lo que es crucial para una gestión eficiente de la cartera.</font></div>

<div><font color="deepgray">

**Adaptabilidad y Flexibilidad:** La estrategia se adapta mensualmente en función de los cambios en el mercado y las características de los activos, lo que permite una gestión dinámica de la cartera.</font></div>

<div><font color="deepgray">

**Evaluación Continua:** La estrategia requiere una evaluación y ajuste continuos, lo que ayuda a identificar y capitalizar oportunidades de mercado en tiempo real.</font></div>


# **8. Visualize Portfolio returns and compare to SP500 returns.**

In [ ]:
# Descargar los datos históricos del ETF SPY (que replica el S&P 500) desde Yahoo Finance
spy = yf.download(tickers='SPY',
                  start='2015-01-01',
                  end=dt.date.today())

# Calcular la rentabilidad diaria del SPY y renombrar la columna a 'SPY Buy&Hold'
spy_ret = np.log(spy[['Adj Close']]).diff().dropna().rename({'Adj Close':'SPY Buy&Hold'}, axis=1)

# Fusionar los retornos diarios de la cartera con los retornos del SPY
# Esto permite comparar el rendimiento de la estrategia de la cartera con el del SPY
portfolio_df = portfolio_df.merge(spy_ret,
                                  left_index=True,
                                  right_index=True)

# 'portfolio_df' ahora contiene tanto los retornos de la estrategia de cartera como los del SPY
portfolio_df

[*********************100%%**********************]  1 of 1 completed


,Strategy Return,SPY Buy&Hold
2017-11-01,0.001361,0.001321
2017-11-02,0.002909,0.000388
2017-11-03,0.006310,0.003333
2017-11-06,0.003008,0.001546
2017-11-07,0.002704,-0.000695
...,...,...
2023-09-25,0.003641,0.004196
2023-09-26,-0.011470,-0.014800
2023-09-27,0.005189,0.000399
2023-09-28,0.007977,0.005781


In [ ]:
import plotly.graph_objects as go

# Establecer el estilo de Plotly para la visualización
plotly_style = 'ggplot2'

# Calcular el retorno acumulativo de la cartera
portfolio_cumulative_return = np.exp(np.log1p(portfolio_df).cumsum()) - 1

# Crear un objeto de figura de Plotly
fig = go.Figure()

# Añadir la serie de datos de la estrategia de la cartera al gráfico
fig.add_trace(go.Scatter(x=portfolio_cumulative_return.index,
                         y=portfolio_cumulative_return['Strategy Return'],
                         mode='lines',
                         name='Strategy Return'))

# Añadir la serie de datos del SPY al gráfico
fig.add_trace(go.Scatter(x=portfolio_cumulative_return.index,
                         y=portfolio_cumulative_return['SPY Buy&Hold'],
                         mode='lines',
                         name='SPY Buy&Hold'))

# Actualizar el título y los ejes del gráfico
fig.update_layout(title='Unsupervised Learning Trading Strategy Returns Over Time',
                  xaxis_title='Date',
                  yaxis_title='Cumulative Return',
                  yaxis_tickformat='%',  # Formato de porcentaje para el eje y
                  template=plotly_style)

# Mostrar el gráfico
fig.show()

<div><font color="deepgray">

*Visualización de los Rendimientos de la Cartera y Comparación con los Rendimientos del SP500*</font></div>

<div><font color="deepgray">

## **1. Descarga de Datos del SP500 y Cálculo de Rendimientos**</font></div>

<div><font color="deepgray">

**Qué es:** Descargamos los datos del índice SP500 (SPY) y calculamos los rendimientos diarios ajustados al cierre para compararlos con los rendimientos de nuestra cartera.</font></div>

<div><font color="deepgray">

**Por qué es importante:** Comparar los rendimientos de nuestra cartera con un benchmark como el SP500 nos permite evaluar el rendimiento relativo y la eficacia de nuestra estrategia de trading.</font></div>

<div><font color="deepgray">

## **2. Fusión de Rendimientos de la Cartera y el SP500**</font></div>

<div><font color="deepgray">

**Qué es:** Fusionamos los rendimientos diarios de nuestra cartera con los del SP500 para realizar una comparación directa.</font></div>

<div><font color="deepgray">

**Por qué es importante:** Esta comparación nos ayuda a entender si nuestra estrategia está superando, igualando o quedando por debajo del rendimiento del mercado en general.</font></div>

<div><font color="deepgray">

## **3. Visualización de Rendimientos Acumulados**</font></div>

<div><font color="deepgray">

**Qué es:** Visualizamos los rendimientos acumulados tanto de la cartera como del SP500 para observar la evolución del rendimiento a lo largo del tiempo.</font></div>

<div><font color="deepgray">

**Por qué es importante:** Esta visualización proporciona una perspectiva clara de cómo la cartera ha crecido en comparación con el mercado, destacando períodos de sobre o subrendimiento.</font></div>

<div><font color="deepgray">

## **4. Análisis y Conclusiones**</font></div>

<div><font color="deepgray">

**Qué es:** Analizamos los resultados de la comparación para sacar conclusiones sobre la efectividad de nuestra estrategia de trading.</font></div>

<div><font color="deepgray">

**Por qué es importante:** Este análisis nos permite ajustar nuestra estrategia, identificar fortalezas y debilidades, y tomar decisiones informadas para futuras inversiones.</font></div>

<div><font color="deepgray">

## **Importancia de Este Paso**</font></div>

<div><font color="deepgray">

**Evaluación del Rendimiento:** La comparación con el SP500 es un estándar de la industria para evaluar el rendimiento de una estrategia de inversión.</font></div>

<div><font color="deepgray">

**Toma de Decisiones Informadas:** Los insights obtenidos de esta comparación son fundamentales para tomar decisiones informadas sobre la gestión de la cartera y la estrategia de inversión.</font></div>

<div><font color="deepgray">

**Ajuste de Estrategia:** Basándonos en esta comparación, podemos ajustar nuestra estrategia para mejorar el rendimiento o reducir el riesgo en el futuro.</font></div>

<div><font color="deepgray">

## **Visualización de los Resultados**</font></div>

<div><font color="deepgray">

**Qué Estamos Visualizando:** La gráfica muestra los rendimientos acumulados de nuestra cartera en comparación con el SP500, destacando cómo nuestra estrategia ha rendido en relación con el mercado en general.</font></div>

<div><font color="deepgray">

**Por qué es Importante:** Esta visualización es crucial para entender el impacto real de nuestra estrategia de trading en términos de rendimiento y riesgo, en comparación con una inversión pasiva en el mercado.</font></div>
